In [1]:
from limes_x.solver import DependencySolver, Dependency, _hashed_dependency

In [2]:
_ids = 0
_ids_map = {}
def make_node(name, ins=None, outs=None):
    if ins is None or outs is None:
        ri, ro = name.split(".")
        ins = {c for c in ri}
        outs = {c for c in ro}

    global _ids, _ids_map
    _ids += 1
    _ids_map[_ids] = name
    e = Dependency(set(ins), set(outs), reference=None)
    he = _hashed_dependency(_ids, e)
    e.reference = he
    return he

def _test(name, nodes: list[_hashed_dependency], given: set[str], targets:set[str], expected: list):
    print(name)

    s = DependencySolver([n.ref for n in nodes])
    result = s.Solve(given, targets)
    if result is None or result == False: result = []

    have = given
    failed = False
    for r in result:
        if not r.requires.issubset(have):
            print(r, have)
            failed = True
        have = have.union(r.produces)
    if not targets.issubset(have):
        failed = True

    result = [n.reference for n in result]
    if failed:
        print(f'ex: {[x for x in expected]}')
        print(f'ac: {[x for x in result]}')
    else:
        print(s.iteration_count)
    # print(G._c)
    # print()
    # Transform._instances.clear()
    # return result, expected

In [3]:
_test(
    'one',[
        make_node('ax', {'a'}, {'x'}),
        make_node('ay', {'a'}, {'y'}),
        make_node('jx', {'j'}, {'x'}),
    ],
    {'a'}, {'x'},
    [
        make_node('ax', {'a'}, {'x'}),
    ],
)

_test(
    'line',[
        make_node('ab', {'a'}, {'b'}),
        make_node('ay', {'a'}, {'y'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('cx', {'c'}, {'x'}),
        make_node('jx', {'j'}, {'x'}),
    ],
    {'a'}, {'x'},
    [
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('cx', {'c'}, {'x'}),
    ],
)

_test(
    'asymm diamond',[
        make_node('sss', {'s'}, {'ss'}),
        make_node('ssa', {'ss'}, {'a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('ay', {'a'}, {'y'}),

        make_node('a_c', {'a'}, {'_c'}),
        # Node('bc.x', {'b', 'c'}, {'x'}),
        make_node('_cc.x', {'_c', 'c'}, {'x'}),
        make_node('jx', {'j'}, {'x'}),
    ],
    {'s'}, {'x'},
    [
        make_node('sss', {'s'}, {'ss'}),
        make_node('ssa', {'ss'}, {'a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('a_c', {'a'}, {'_c'}),
        make_node('_cc.x', {'_c', 'c'}, {'x'}),
    ],
)


_test(
    'loop',[
        make_node('ca', {'c'}, {'a'}),
        make_node('sa', {'s'}, {'a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('ay', {'a'}, {'y'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('cx', {'c'}, {'x'}),
        make_node('jx', {'j'}, {'x'}),
    ],
    {'s'}, {'x'},
    [
        make_node('sa', {'s'}, {'a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('cx', {'c'}, {'x'}),
    ],
)

_test(
    'loop b',[
        make_node('sa', {'s'}, {'a'}),
        make_node('ca', {'c'}, {'a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('ay', {'a'}, {'y'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('cx', {'c'}, {'x'}),
        make_node('jx', {'j'}, {'x'}),
    ],
    {'s'}, {'x'},
    [
        make_node('sa', {'s'}, {'a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('cx', {'c'}, {'x'}),
    ],
)

_test(
    'loop vs fork',[
        make_node('sa', {'s'}, {'a'}),
        make_node('ab+', {'a'}, {'b', 'j', 'k'}),
        make_node('bk.c', {'b', 'k'}, {'c'}),
        make_node('c.x', {'c'}, {'x'}),
        make_node('ja', {'j'}, {'a'}),
    ],
    {'s'}, {'x'},
    [        
        make_node('sa', {'s'}, {'a'}),
        make_node('ab+', {'a'}, {'b', 'j', 'k'}),
        make_node('bk.c', {'b', 'k'}, {'c'}),
        make_node('c.x', {'c'}, {'x'}),
    ],
)

# {'+2'}
# {sa, ab, bc, _ab, _ay, _ac, +1, +3, end}
# {'j', '+1', '_y', 'b', 'x', 'c', '+x', '+3', 'a', '_a', '_c', '_b'}}
_test(
    'complex',[
        make_node('sa', {'s'}, {'a', '_a'}),

        make_node('ab', {'a'}, {'b'}),
        make_node('ay', {'a'}, {'y'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('ca', {'c'}, {'a'}),
        make_node('cx', {'c'}, {'x'}),
        make_node('jx', {'j'}, {'x'}),

        make_node('_ab', {'_a'}, {'_b'}),
        make_node('_ay', {'_a'}, {'_y'}),
        make_node('_ac', {'_a'}, {'_c'}),
        make_node('_bc.x', {'_b', '_c'}, {'_x'}),
        make_node('_jx', {'_j'}, {'_x'}),

        make_node('+1', {'b', '_b'}, {'+1'}),
        make_node('+2', {'c', '_b', 'a'}, {'+2', 'a'}),
        make_node('+3', {'+1', '+2', '_y', '_c', 'c'}, {'j', '+3', 'x'}),

        make_node('end', {'+1', '+2', '+3'}, {'+x'})
    ],
    {'s'}, {'+x'},
    [
        make_node('sa', {'s'}, {'a', '_a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c'}),

        make_node('_ab', {'_a'}, {'_b'}),
        make_node('_ac', {'_a'}, {'_c'}),
        make_node('_ay', {'_a'}, {'_y'}),

        make_node('+1', {'b', '_b'}, {'+1'}),
        make_node('+2', {'c', '_b', 'a'}, {'+2', 'a'}),
        make_node('+3', {'+1', '+2', '_y', '_c', 'c'}, {'+2', 'j', '+3', 'x'}),

        make_node('end', {'+1', '+2', '+3'}, {'+x'})
    ],
)

_test(
    'distributed',[
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c1'}),

        make_node('2', {'c1'}, {'c2'}),
        make_node('3', {'s1'}, {'c3'}),

        make_node('g', {'a', 'b', 'c1', 'c2', 'c3'}, {'g1', 'g2'}),

        make_node('*', {'c1', 'c2', 'c3', 's2', 'g1', 'g2'}, {'d'}),

        make_node('de', {'d'}, {'e'}),
        make_node('ex', {'e'}, {'x'})
    ],
    {'a', 's1', 's2'}, {'x'},
    [
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c1'}),

        make_node('2', {'c1'}, {'c2'}),
        make_node('3', {'s1'}, {'c3'}),
        
        make_node('g', {'a', 'b', 'c1', 'c2', 'c3'}, {'g1', 'g2'}),

        make_node('*', {'c1', 'c2', 'c3', 's2'}, {'d'}),

        make_node('de', {'d'}, {'e'}),
        make_node('ex', {'e'}, {'x'})
    ],
)

_test(
    'overlay',[
        make_node('s.abc', {'s'}, {'a', 'b', 'c'}),
        make_node('ab.jk', {'a', 'b'}, {'j', 'k'}),
        make_node('bc.ki', {'b', 'c'}, {'k', 'i'}),
        make_node('ca.ij', {'a', 'c'}, {'i', 'j'}),

        make_node('end', {'i', 'j', 'k'}, {'x'})
    ],
    {'s'}, {'x'},
    [
        make_node('s.abc', {'s'}, {'a', 'b', 'c'}),
        make_node('ab.jk', {'a', 'b'}, {'j', 'k'}),
        make_node('bc.ki', {'b', 'c'}, {'k', 'i'}),
        make_node('ca.ij', {'a', 'c'}, {'i', 'j'}),

        make_node('end', {'i', 'j', 'k'}, {'x'})
    ],
)

_test(
    'should pick shorter path',[
        make_node('s.ai'),
        
        make_node('a.b'),
        make_node('b.x'),

        make_node('i.j'),
        make_node('j.k'),
        make_node('k.x'),
    ],
    {"s"}, {"x"},
    [
        make_node('s.ai'),
        make_node('a.b'),
        make_node('b.x'),
    ],
)

one
2
line
4
asymm diamond
7
loop
6
loop b
6
loop vs fork
7
complex
14
distributed
8
overlay
8
should pick shorter path
6
